## NOTEBOOK PARA APRENDER IDIOMAS

Para aprender idiomas, en este caso el turco, la web de [www.turkishclass101.com](www.turkishclass101.com) propone aprender cada día una palabra nueva y usar dicha palabra en un par de frases. Además de la palabra en turco y dichas frases, también aparece su traducción.

Vamos a scrapear la web yendo día a día a lo largo de un periodo de tiempo que nosotros determinemos para poder sacar un archivo Excel con una tabla donde aparezcan la lista de palabras, su traducción y las frases donde se usan (con su traducción igualmente).

Vamos a hacer este scraping usando la librería [autoscraper](https://github.com/alirezamika/autoscraper), que facilita el web scraping al aprender las reglas del scraping de una URL dada y devuelve los elementos con una estructura similar de esa URL u otras URLs con contenido similar.

In [3]:
# Importamos librerías

# Librerías estándar
import pandas as pd
import numpy as np
import datetime

# Librería para medir el tiempo de ejecución
import time

In [ ]:
# Librería para hacer el scraping

# Si no la tenemos instalada: !pip install autoscraper
from autoscraper import AutoScraper

### TEST: Scrapear el día de hoy 

Antes de lanzar el código completo, pruebo a scrapear un solo día para ver como son los outputs que obtendré.

In [6]:
# Le hago el Autoscraper al día de hoy
# y luego le pido que busque similares en los días anteriores

# Defino la URL correspondiente a un día en concreto
url_inicial = 'https://www.turkishclass101.com/turkish-phrases/02142021#wotd-widget'

# Defino unas palabras de ejemplo
palabra_tr = ['indirim']
traduccion_en = ['sale']
frases_tr = ['Ayakkabı bölümünde büyük bir indirimli satış var.']
frases_en = ['There is a big sale in the shoe department.']

# Inicializo los elementos que quiero scrapear
scraper_palabra_tr = AutoScraper()
scraper_traduccion_en = AutoScraper()
scraper_frases_tr = AutoScraper()
scraper_frases_en = AutoScraper()

# Construyo la estructura de los elementos a scrapear
result_palabra_tr = scraper_palabra_tr.build(url_inicial, palabra_tr)
result_traduccion_en = scraper_traduccion_en.build(url_inicial, traduccion_en)
result_frases_tr = scraper_frases_tr.build(url_inicial, frases_tr)
result_frases_en = scraper_frases_en.build(url_inicial, frases_en)

print('Palabra en turco:',result_palabra_tr)
print('Traducción al inglés:',result_traduccion_en)
print('Frases en turco:',result_frases_tr)
print('Traducción al inglés:',result_frases_en)

Palabra en turco: ['indirim']
Traducción al inglés: ['sale']
Frases en turco: ['Ayakkabı bölümünde büyük bir indirimli satış var.', 'indirimli bir satışa bakmak', 'yüzde elli indirimli satış']
Traducción al inglés: ['There is a big sale in the shoe department.', 'look for a sale', 'fifty percent off sale']


### AUTOMATIZACIÓN DEL PROCESO

Primero separamos por pasos y luego juntaremos el código en una [función](#funcion) que recibirá como input el intervalo de días para el que queremos sacar las palabras. 

#### PASO 1: Seleccionar el intervalo temporal

In [8]:
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2020, 12, 31)
delta = datetime.timedelta(days=1)

print('Tenemos datos que van desde el %s hasta el %s.' %(start_date, end_date))

lista_fechas = []

while start_date <= end_date:
    # Convertimos las fechas al formato con el que entran en la URL
    lista_fechas.append(start_date.strftime("%m%d%Y"))
    start_date += delta

Tenemos datos que van desde el 2020-01-01 hasta el 2020-12-31.


#### PASO 2: Scrapear las fechas seleccionadas

In [10]:
lista_palabra_tr = []
lista_trad_en = []
lista_frases_tr = []
lista_frases_en = []
lista_fecha = []


start_time = time.time()

for i in range(len(lista_fechas)):
    
    # Hago print para ver el estado del loop cada 30 dias (aprox cada mes)
    if i%1==0:
        print("-",end="")
    if i%30==0:
        print(' %d DE %d' %(i, len(lista_fechas)))

    
    url = 'https://www.turkishclass101.com/turkish-phrases/'+lista_fechas[i]+'#wotd-widget'
    
    # Primero compruebo que hay el mismo número de frases en turco que en inglés
    frases_tr = scraper_frases_tr.get_result_similar(url)
    frases_en = scraper_frases_en.get_result_similar(url)

    if len(frases_tr) == len(frases_en):
        
        # Si hay más de una frase, tengo que duplicar n veces la palabra en turco y en inglés
        fechaN = [lista_fechas[i]]*len(frases_tr)
        
        palabra_tr = scraper_palabra_tr.get_result_similar(url)
        palabra_trN = palabra_tr*len(frases_tr)
        
        trad_en = scraper_traduccion_en.get_result_similar(url)
        trad_enN = trad_en*len(frases_tr)
        
        # Lo agrupamos en listas
        lista_fecha.append(fechaN)
        lista_palabra_tr.append(palabra_trN)
        lista_trad_en.append(trad_enN)
        lista_frases_tr.append(frases_tr)
        lista_frases_en.append(frases_en)
              
        
    else:
        break
        
# Desnidamos las listas de listas
flat_lista_fecha = [item for sublist in lista_fecha for item in sublist]
flat_lista_palabra_tr = [item for sublist in lista_palabra_tr for item in sublist]
flat_lista_trad_en = [item for sublist in lista_trad_en for item in sublist]
flat_lista_frases_tr = [item for sublist in lista_frases_tr for item in sublist]
flat_lista_frases_en = [item for sublist in lista_frases_en for item in sublist]
        
print('\nEl proceso ha tardado %.1f minutos.' %((time.time()-start_time)/60))

- 0 DE 366
------------------------------ 30 DE 366
------------------------------ 60 DE 366
------------------------------ 90 DE 366
------------------------------ 120 DE 366
------------------------------ 150 DE 366
------------------------------ 180 DE 366
------------------------------ 210 DE 366
------------------------------ 240 DE 366
------------------------------ 270 DE 366
------------------------------ 300 DE 366
------------------------------ 330 DE 366
------------------------------ 360 DE 366
-----
El proceso ha tardado 6.7 minutos.


#### PASO 3: Crear un dataframe con los resultados

In [11]:
df = pd.DataFrame(columns=['Dia','Palabra_tr','Traduccion_en','Frase_tr','Frase_en'])
df['Dia'] = pd.to_datetime(flat_lista_fecha, format="%m%d%Y")
df['Palabra_tr'] = flat_lista_palabra_tr
df['Traduccion_en'] = flat_lista_trad_en
df['Frase_tr'] = flat_lista_frases_tr
df['Frase_en'] = flat_lista_frases_en

In [12]:
df

,Dia,Palabra_tr,Traduccion_en,Frase_tr,Frase_en
0,2020-01-01,karıştırmak,mix,"Suyu ekle, ısıt ve karıştır.","Mix with water, heat and stir."
1,2020-01-01,karıştırmak,mix,Unu suyla karıştırmalısın.,You need to mix the flour with water.
2,2020-01-01,karıştırmak,mix,suyla karıştırmak,mix with water
3,2020-01-02,üniversite,university,O ünlü bir üniversiteden tam burs kazandı.,He received a full scholarship from a famous u...
4,2020-01-02,üniversite,university,eski üniversite,old university
...,...,...,...,...,...
1181,2020-12-30,biyoloji,biology,"Biyoloji, yaşayan canlıların incelenmesidir.",Biology is the study of living organisms.
1182,2020-12-30,biyoloji,biology,Biyoloji sınıfı,biology class
1183,2020-12-31,İtalya,Italy,"Antik Roma İmparatorluğu`nun kalbi, modern I...",The heart of the ancient Roman empire was mode...
1184,2020-12-31,İtalya,Italy,İtalya haritası,map of Italy


#### PASO 4: Guardamos el csv

In [13]:
df.to_csv('df_turco2020.csv', index=False)

<a id='funcion'></a>
##### DEFINIMOS LA FUNCIÓN QUE ENGLOBA LOS PASOS ANTERIORES

In [21]:
def scraping_turco(fecha_inicial, fecha_final):
    
    #PASO 1: Seleccionar el intervalo temporal
    start_date = datetime.datetime.strptime(fecha_inicial, '%d-%m-%Y').date()
    end_date = datetime.datetime.strptime(fecha_final, '%d-%m-%Y').date()
    delta = datetime.timedelta(days=1)
    
    lista_fechas = []
    while start_date <= end_date:
        # Convertimos las fechas al formato con el que entran en la URL
        lista_fechas.append(start_date.strftime("%m%d%Y"))
        start_date += delta
        
        
    # PASO 2: Scrapear las fechas seleccionadas
    lista_palabra_tr = []
    lista_trad_en = []
    lista_frases_tr = []
    lista_frases_en = []
    lista_fecha = []

    start_time = time.time()

    for i in range(len(lista_fechas)):
        
        # Hago print para ver el estado del loop cada 30 dias (aprox cada mes)
        if i%1==0:
            print("-",end="")
        if i%30==0:
            print(' %d DE %d' %(i, len(lista_fechas)))

        url = 'https://www.turkishclass101.com/turkish-phrases/'+lista_fechas[i]+'#wotd-widget'

        # Primero compruebo que hay el mismo número de frases en turco que en inglés
        frases_tr = scraper_frases_tr.get_result_similar(url)
        frases_en = scraper_frases_en.get_result_similar(url)

        if len(frases_tr) == len(frases_en):

            # Si hay más de una frase, tengo que duplicar n veces la palabra en turco y en inglés
            fechaN = [lista_fechas[i]]*len(frases_tr)

            palabra_tr = scraper_palabra_tr.get_result_similar(url)
            palabra_trN = palabra_tr*len(frases_tr)

            trad_en = scraper_traduccion_en.get_result_similar(url)
            trad_enN = trad_en*len(frases_tr)

            # Lo agrupamos en listas
            lista_fecha.append(fechaN)
            lista_palabra_tr.append(palabra_trN)
            lista_trad_en.append(trad_enN)
            lista_frases_tr.append(frases_tr)
            lista_frases_en.append(frases_en)


        else:
            break

    # Desnidamos las listas de listas
    flat_lista_fecha = [item for sublist in lista_fecha for item in sublist]
    flat_lista_palabra_tr = [item for sublist in lista_palabra_tr for item in sublist]
    flat_lista_trad_en = [item for sublist in lista_trad_en for item in sublist]
    flat_lista_frases_tr = [item for sublist in lista_frases_tr for item in sublist]
    flat_lista_frases_en = [item for sublist in lista_frases_en for item in sublist]
    print('\nEl proceso ha tardado %.1f minutos.' %((time.time()-start_time)/60))
    
    
    # PASO 3: Crear un dataframe con los resultados
    df = pd.DataFrame(columns=['Dia','Palabra_tr','Traduccion_en','Frase_tr','Frase_en'])
    df['Dia'] = pd.to_datetime(flat_lista_fecha, format="%m%d%Y")
    df['Palabra_tr'] = flat_lista_palabra_tr
    df['Traduccion_en'] = flat_lista_trad_en
    df['Frase_tr'] = flat_lista_frases_tr
    df['Frase_en'] = flat_lista_frases_en
    
    
    # PASO 4: Guardamos el csv
    df.to_csv('df_turco2020.csv', index=False)

In [22]:
scraping_turco('01-02-2021', '10-02-2021')

- 0 DE 10
---------
El proceso ha tardado 0.2 minutos.
